In [70]:
import sys
sys.path.append('..\\..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
import QCreator.auxiliary_functions as auxfuncs
from matplotlib import pyplot as plt
reload(gdspy)

<module 'gdspy' from 'C:\\Users\\avasi\\anaconda3\\lib\\site-packages\\gdspy\\__init__.py'>

In [71]:
tl_core = 11.
tl_gap = 6.
tl_ground = 20.

In [72]:
pad_offset = 1100
pad_element_offset = 400



layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':3,
    'air bridges':2,
    'air bridge pads':1,
    'air bridge sm pads':4,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17
}

sample = creator.Sample('Res-test-16-bridges',layers_configuration, epsilon=11.45)


airbridge = elements.AirBridgeGeometry(pad_width = 30,
                                       pad_length = 30,
                                       narrow_width = 10,
                                       narrow_length = 56,
                                       pad_distance = 36,
                                       sm_pad_length = 10,
                                       sm_pad_distance = 56,
                                       layer_configuration = sample.layer_configuration,
                                       bridge_style='misis')

# airbridge = elements.AirBridgeGeometry(pad_width = 36,
#                                        pad_length = 22,
#                                        pad_distance = 62,
#                                        narrow_width = 20,
#                                        narrow_length = 46,
#                                        sm_pad_length = 10,
#                                        sm_pad_distance = 70,
#                                        layer_configuration = sample.layer_configuration)

In [73]:
#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=4.7e3
sample.chip_geometry.sample_horizontal_size=9.7e3
central_line_y = sample.chip_geometry.sample_vertical_size/2
chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads, 800)
sample.add(chip_edge_ground)

#grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
#                                  width=4, period=50)
#sample.add(grid_ground)

In [74]:
# 1. Create contact pads:
pads_left = []
pads_right = []
for pad_side_id in range(1):
    pad = elements.Pad('pad-left-' + str(pad_side_id),
                       (pad_offset, sample.chip_geometry.sample_vertical_size / 2), np.pi, tl_core,
                       tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_left.append(pad)
    sample.add(pad)
    pad = elements.Pad('pad-right-' + str(pad_side_id),
                       (sample.chip_geometry.sample_horizontal_size - pad_offset,
                        sample.chip_geometry.sample_vertical_size / 2), 0, tl_core,
                       tl_gap, tl_ground,
                       layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                       **elements.default_pad_geometry())
    pads_right.append(pad)
    sample.add(pad)

pad0 = pads_left[0]
pad1 = pads_right[0]

In [75]:
#sample.draw_design()
#sample.watch()

In [76]:
#pad0.get_terminals()


In [77]:
pad1.get_terminals()

{'port': Type: cpw, Position: (8600.0, 2350.0), Orientation: 0
 w: 11.0, s: 6.0, g: 20.0
 reversed: True}

In [78]:
#sample.connect_cpw(pad0, pad1, 'port', 'port', 'name', points = [])

In [79]:
initial_spacing = 200

coupler_length_1 = 280
coupler_start_1 = [pad_offset + initial_spacing,
                 sample.chip_geometry.sample_vertical_size / 2]
# coupler_start_1 = [sample.chip_geometry.sample_horizontal_size / 9 + coupler_length_1/2,
#                  sample.chip_geometry.sample_vertical_size / 2]

g1,g2, par1, par2= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_1[0],
                                                  coupler_start_1[1],
                                                  coupler_length_1,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 2900,
                                                  length_left1 = 150,
                                                  length_right1 = 150,
                                                  closed_end_meander_length2 = 3000,
                                                  length_left2 = 150,
                                                  length_right2 = 150,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)
coupler_length_2 = 300
coupler_start_2 = [pad_offset + 1 * (sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                 sample.chip_geometry.sample_vertical_size / 2]

g3,g4, par3, par4= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_2[0],
                                                  coupler_start_2[1],
                                                  coupler_length_2,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 3110,
                                                  length_left1 = 200,
                                                  length_right1 = 200,
                                                  closed_end_meander_length2 = 3250,
                                                  length_left2 = 200,
                                                  length_right2 = 200,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)


coupler_length_3 = 330
coupler_start_3 = [pad_offset + 2 * (sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                 sample.chip_geometry.sample_vertical_size / 2]

g5,g6, par5, par6= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_3[0],
                                                  coupler_start_3[1],
                                                  coupler_length_3,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 3400,
                                                  length_left1 = 200,
                                                  length_right1 = 200,
                                                  closed_end_meander_length2 = 3600,
                                                  length_left2 = 200,
                                                  length_right2 = 200,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)
coupler_length_4 = 365
coupler_start_4 = [pad_offset + 3 *(sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                 sample.chip_geometry.sample_vertical_size / 2]

g7,g8, par7, par8= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_4[0],
                                                  coupler_start_4[1],
                                                  coupler_length_4,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 3680,
                                                  length_left1 = 200,
                                                  length_right1 = 200,
                                                  closed_end_meander_length2 = 3850,
                                                  length_left2 = 200,
                                                  length_right2 = 200,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)
coupler_length_5 = 390
coupler_start_5 = [pad_offset + 4 * (sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                 sample.chip_geometry.sample_vertical_size / 2]

g9,g10, par9, par10= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_5[0],
                                                  coupler_start_5[1],
                                                  coupler_length_5,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 4000,
                                                  length_left1 = 200,
                                                  length_right1 = 200,
                                                  closed_end_meander_length2 = 4250,
                                                  length_left2 = 200,
                                                  length_right2 = 200,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)


coupler_length_6 = 420
coupler_start_6 = [pad_offset + 5 * (sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                 sample.chip_geometry.sample_vertical_size / 2]

g11,g12, par11, par12= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_6[0],
                                                  coupler_start_6[1],
                                                  coupler_length_6,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 4450,
                                                  length_left1 = 200,
                                                  length_right1 = 350,
                                                  closed_end_meander_length2 = 4730,
                                                  length_left2 = 350,
                                                  length_right2 = 200,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)

coupler_length_7 = 460
coupler_start_7 = [pad_offset + 6 *(sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                 sample.chip_geometry.sample_vertical_size / 2]

g13,g14, par13, par14= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_7[0],
                                                  coupler_start_7[1],
                                                  coupler_length_7,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 5000,
                                                  length_left1 = 300,
                                                  length_right1 = 150,
                                                  closed_end_meander_length2 = 5300,
                                                  length_left2 = 150,
                                                  length_right2 = 300,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)

coupler_length_8 = 520
coupler_start_8 = [pad_offset + 7*(sample.chip_geometry.sample_horizontal_size - 2*pad_offset)/8,
                   sample.chip_geometry.sample_vertical_size / 2]
# coupler_start_8 = [8 * sample.chip_geometry.sample_horizontal_size / 9 - coupler_length_8,
#                   sample.chip_geometry.sample_vertical_size / 2]

g15,g16, par15, par16= auxfuncs.draw_double_resonator(sample,
                                                  coupler_start_8[0],
                                                  coupler_start_8[1],
                                                  coupler_length_8,
                                                  resonator_core = 7,
                                                  resonator_gap = 8,
                                                  resonator_ground = 20,
                                                  tl_core = 11,
                                                  tl_gap = 6,
                                                  tl_ground = 20,
                                                  grounding_width = 10,
                                                  closed_end_meander_length1 = 5600,
                                                  length_left1 = 400,
                                                  length_right1 = 150,
                                                  closed_end_meander_length2 = 6000,
                                                  length_left2 = 150,
                                                  length_right2 = 400,
                                                  open_end_length1 = 500,
                                                  open_end_length2 = 500,
                                                  port_orientation='left',
                                                  airbridge=airbridge,
                                                  min_bridge_spacing_closed_end=100,
                                                  min_bridge_spacing_open_end=150,
                                                  meander_r=55)


sample.connect_cpw(pad0, g1, port1='port', port2='narrow', name='right TL', points=[],
                   airbridge=airbridge, min_spacing=150)

sample.connect_cpw(g2, g3, port1='narrow', port2='narrow', name='right TL', points=[],
                   airbridge=airbridge, min_spacing=150)
sample.connect_cpw(g4, g5, port1='narrow', port2='narrow', name='right TL', points=[], 
                   airbridge=airbridge, min_spacing=150)
sample.connect_cpw(g6, g7, port1='narrow', port2='narrow', name='right TL', points=[], 
                   airbridge=airbridge, min_spacing=150)
sample.connect_cpw(g8, g9, port1='narrow', port2='narrow', name='right TL', points=[], 
                   airbridge=airbridge, min_spacing=150)
sample.connect_cpw(g10, g11, port1='narrow', port2='narrow', name='right TL', points=[],
                   airbridge=airbridge, min_spacing=150)
sample.connect_cpw(g12, g13, port1='narrow', port2='narrow', name='right TL', points=[],
                   airbridge=airbridge, min_spacing=150)
sample.connect_cpw(g14, g15, port1='narrow', port2='narrow', name='right TL', points=[], 
                   airbridge=airbridge, min_spacing=150)





sample.connect_cpw(g16, pad1, port1='narrow', port2='port', name='right TL', points=[])
                   #airbridge=airbridge, min_spacing=150)

[CPW "right TL", l=384.500]

In [80]:
sample.draw_design()
sample.watch()

In [81]:
sys, connections, elements_ = sample.get_tls(cutoff=2e11)
#f, delta, modes = sys.get_modes()
#num_modes=10

#(res_fr,kappa,_)=auxfuncs.search_for_resonators_qubits(f,delta,0,15)[:num_modes]

In [82]:
sys.create_dynamic_equation_matrices()[0].shape

(6276, 6276)

In [ ]:
f, delta, modes = sys.get_modes()

In [145]:
num_modes=16
(res_fr,kappa,_)=auxfuncs.search_for_resonators_qubits(f,delta,0,10)[:num_modes]
#f[:num_modes]/(2e6*np.pi), delta[:num_modes]/(2e6*np.pi), f/(2*np.pi*2*delta)

Resonance frequencies are, GHz: [7.77368208 7.98039795]
Kappas are, us^-1: [0.4525602  0.52268488]
Quality factors are: [107927.04529418  95932.21730358]


In [407]:
f_ = np.linspace(3e9, 10e9, 1601)

In [ ]:
s21 = sample.get_s21('pad-left-0', 'pad-right-0', f_)
#plt.plot(f_, s21)


In [ ]:
plt.plot(f_, abs(s21))

In [83]:
sample.write_to_gds()

Gds file has been writen here:  C:\Users\avasi\Documents\GitHub\QCreator\designs\Resonators_test\Res-test-16-bridges.gds


In [ ]:
from math import sqrt
l = 3400e-6
f0 = (3e8/sqrt(11.45)) * 1/(4*l)
f0/1e9

In [ ]:
f_bridge = [
    4.1424458,
    4.59551078,
    4.88016854,
    5.51433467,
    5.94025965,
    6.90750097,
    7.1425325,
    7.69999696,
    7.9779454,
    8.68035539
]

f_nobridge = [
    4.18101758,
    4.63208409,
    4.92084733,
    5.55735695,
    5.99094905,
    6.96159938,
    7.19772259,
    7.75222642,
    8.04561973,
    8.7448298
]

numbers = np.arange(10)

lengths = [
    3350,
    3650,
    3790,
    4090,
    4230,
    4930,
    5320,
    6020,
    6400,
    7100
]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)


fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("№", fontsize=14)
ax.set_ylabel("frequency", fontsize=14)
ax.grid(which="major", linewidth=1.2)
ax.grid(which="minor", linestyle="--", color="gray", linewidth=0.5)
ax.scatter(numbers,f_bridge, c = 'red', label = 'bridges')
ax.scatter(numbers,f_nobridge, c = 'blue', label = 'w/o bridges')
ax.legend()

#ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='major', length=10, width=2)
ax.tick_params(which='minor', length=5, width=1)
plt.savefig('demo.png', transparent=True)

In [ ]:
diff = np.empty(len(f_bridge))
diff = np.array(f_nobridge) - np.array(f_bridge)
diff

In [ ]:
plt.plot(numbers, diff)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("№", fontsize=14)
ax.set_ylabel("frequency", fontsize=14)
ax.grid(which="major", linewidth=1.2)
ax.grid(which="minor", linestyle="--", color="gray", linewidth=0.5)
ax.scatter(numbers,diff, c = 'red', label = 'difference')
#ax.scatter(numbers,f_nobridge, c = 'blue', label = 'bridges')
ax.legend()

#ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='major', length=10, width=2)
ax.tick_params(which='minor', length=5, width=1)
plt.savefig('demo1.png', transparent=True)

In [ ]:
plt.plot(lengths, diff)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_xlabel("length", fontsize=14)
ax.set_ylabel("difference", fontsize=14)
ax.grid(which="major", linewidth=1.2)
ax.grid(which="minor", linestyle="--", color="gray", linewidth=0.5)
ax.scatter(lengths,diff, c = 'red', label = 'difference')
#ax.scatter(numbers,f_nobridge, c = 'blue', label = 'bridges')
ax.legend()

#ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='major', length=10, width=2)
ax.tick_params(which='minor', length=5, width=1)
plt.savefig('demo1.png', transparent=True)

In [408]:
import resonator_tools
from resonator_tools.circuit import notch_port, reflection_port

ModuleNotFoundError: No module named 'resonator_tools'